In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import gensim
from gensim.corpora import Dictionary
import joblib
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

SOME_FREE_SONGTEXT = "So one night I said to Kathy We gotta get away somehow Go somewhere south and somewhere warm But for God's sake let's go now. And Kathy she sort of looks at me And asks where I wanna go So I look back and I hear me say I don't care but we gotta go chorus and key change And all the other people Who slepwalk thru their days Just sort of faded out of sight When we two drove away And ev'ry day we travelled We were lookin' to get wise And we learned what was the truth And we learned what were the lies And in LA we bought a bus Sort of old and not too smart So for six hundred and fifty bucks We got out and made a start We hit the road down to the South And drove into Mexico That old bus was some old wreck But it just kept us on the road. chorus etc We drove up to Alabam And a farmer gave us some jobs We worked them crops all night and day And at night we slept like dogs We got paid and Kathy said to me It's time to make a move again And when I looked into her eyes I saw more than a friend. chorus etc And now we've stopped our travels And we sold the bus in Texas And we made our home in Austin And for sure it ain't no palace And Kathy and me we settled down And now our first kid's on the way Kathy and me and that old bus We did real good to get away."

In [7]:
def load_songs(filename, nrows=None):
        return pd.read_csv(filename, sep='\t', nrows=nrows)


def load_artists(filename, nrows=None):
        return pd.read_csv(filename, sep=',', nrows=nrows)

def load_albums(filename, nrows=None):
        return pd.read_csv(filename, sep='\t', nrows=nrows)

def random_key_from_dict(dictionary, seed=123):
        keys_list = list(dictionary.keys())
        np.random.seed(seed)
        random_index = np.random.choice(len(keys_list))
        return keys_list[random_index]

In [8]:
### Quick overview over some columns
df = load_songs('wasabi_songs.csv', nrows=2000)
for property in ['artist', 'title', 'publicationDateAlbum', 'album_genre',
                 'valence', 'arousal', 'valence_predicted', 'arousal_predicted', 'bpm']:
    print('Histogram for:', property)
    print(pd.value_counts(df[property]), '\n\n')

Histogram for: artist
artist
A Cor Do Som            107
A                       102
A Global Threat         100
A Halo Called Fred       79
A Day To Remember        74
                       ... 
A Christmas Carol         1
A Contrail To Follow      1
A Farewell Rescue         1
A Girl Called Jane        1
A Chilling Silence        1
Name: count, Length: 129, dtype: int64 


Histogram for: title
title
Beleza Pura                           6
Menino Deus                           6
Zanzibar (As Cores)                   6
Abri A Porta                          5
Alto Astral                           5
                                     ..
Take Care                             1
A Song The World Can Sing Out Loud    1
White Lines And Lipstick              1
Shoot From The Hip                    1
First Light Of Dawn                   1
Name: count, Length: 1881, dtype: int64 


Histogram for: publicationDateAlbum
publicationDateAlbum
2008    158
2005    155
2004    131
2007    125
2006  

/tmp/ipykernel_3618/3752516917.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(df[property]), '\n\n')
/tmp/ipykernel_3618/3752516917.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(df[property]), '\n\n')
/tmp/ipykernel_3618/3752516917.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(df[property]), '\n\n')
/tmp/ipykernel_3618/3752516917.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(df[property]), '\n\n')
/tmp/ipykernel_3618/3752516917.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use p

Next Step: Make new df with all relevant features we want to incorporate

In [17]:
print(df.head)
features = df[["title","artist", "album_genre", "valence", "publicationDateAlbum"]]

X = features
y = df["bpm"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)



<bound method NDFrame.head of       Unnamed: 0                                 _id abstract  \
0              0  ObjectId(5714dec325ac0d8aee3804e7)      NaN   
1              1  ObjectId(5714dec325ac0d8aee3804e8)      NaN   
2              2  ObjectId(5714dec325ac0d8aee3804e9)      NaN   
3              3  ObjectId(5714dec325ac0d8aee3804ea)      NaN   
4              4  ObjectId(5714dec325ac0d8aee3804eb)      NaN   
...          ...                                 ...      ...   
1995        1995  ObjectId(5714dec325ac0d8aee380cb2)      NaN   
1996        1996  ObjectId(5714dec325ac0d8aee380cb3)      NaN   
1997        1997  ObjectId(5714dec325ac0d8aee380cb4)      NaN   
1998        1998  ObjectId(5714dec325ac0d8aee380cb5)      NaN   
1999        1999  ObjectId(5714dec325ac0d8aee380cb6)      NaN   

                 albumTitle       album_genre animux_content animux_contents  \
0     How Ace Are Buildings  Alternative Rock            NaN             NaN   
1     How Ace Are Buildings  

In [13]:
def create_model():
 # create model
 model = Sequential()
 model.add(Dense(12, input_dim=8, activation='relu'))
 model.add(Dense(8, activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

seed = 7
np.random.seed(seed)

model = KerasClassifier(model=create_model, epochs=150, batch_size=10, verbose=0)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X_train, y_train, cv=kfold)

ValueError: Input y contains NaN.